## Reduced without AMPA, Wong and Wang 2006

In [ ]:
import numpy as np
import plotly.graph_objects as go

np.random.seed(1234)

dt = 0.5  
t_max = 3000.0
time = np.arange(0, t_max + dt, dt) 
n_time_steps = len(time)

# 模型参数
tau_NMDA = 100.0  # NMDA 突触时间常数 (ms)
tau_AMPA = 2.0  # AMPA 突触时间常数 (ms)

# Synaptic Efficacies
J_N = 0.2609  # NMDA 再回路激励强度
J_N_cross = 0.0497  # 交叉 NMDA 激励强度

# NMDA Gain Factor
gamma = 0.641  

# 外部输入 
I_0 = 0.3255  # 外部输入基线 (nA)
mu0 = 30  # Hz, 平均输入
J_Input = 5.2e-4  # 输入激励
coherence = 0.032  # 一致性 (0 到 1)

# Population 1 和 2 的刺激输入
I_stim_1 = J_Input * mu0 * (1 + coherence)
I_stim_2 = J_Input * mu0 * (1 - coherence)

stim_start = 500.0  # 刺激开始时间 (ms)
stim_end = 2000.0  # 刺激结束时间 (ms)

sigma_I = 0.02  # 输入噪声的标准差 (nA)

S1 = np.zeros(n_time_steps)  
S2 = np.zeros(n_time_steps)  
r1 = np.zeros(n_time_steps)  
r2 = np.zeros(n_time_steps)  
I1 = np.zeros(n_time_steps)  
I2 = np.zeros(n_time_steps)  


decision_made = False
decision_times = 0.0
choice = 0

# Define transfer function H(I)
a, b, d = 270.0, 108.0, 0.154
def H(I):
    I_eff = a * I - b
    return I_eff / (1.0 - np.exp(-d * I_eff))

# 外部输入
I_ext1 = np.full(n_time_steps, I_0)
I_ext2 = np.full(n_time_steps, I_0)

stim_indices = np.where((time >= stim_start) & (time <= stim_end))[0]
I_ext1[stim_indices] += I_stim_1
I_ext2[stim_indices] += I_stim_2

# 模拟网络动力学
I_noise1, I_noise2 = sigma_I * np.random.randn(), sigma_I * np.random.randn()
S1[0] = S2[0] = 0.1

for t in range(n_time_steps - 1):
    # 计算噪声
    I_noise1 += dt / tau_AMPA * (-I_noise1) + np.sqrt(dt / tau_AMPA) * sigma_I * np.random.randn()
    I_noise2 += dt / tau_AMPA * (-I_noise2) + np.sqrt(dt / tau_AMPA) * sigma_I * np.random.randn()

    # 计算总输入电流
    I1[t] = J_N * S1[t] - J_N_cross * S2[t] + I_ext1[t] + I_noise1
    I2[t] = J_N * S2[t] - J_N_cross * S1[t] + I_ext2[t] + I_noise2

    # 计算放电率
    r1[t] = H(I1[t])
    r2[t] = H(I2[t])
    r1[t] = max(0, r1[t])
    r2[t] = max(0, r2[t])
    
    # 更新突触门控变量
    S1[t + 1] = S1[t] + (-S1[t] / tau_NMDA + (1 - S1[t]) * gamma * r1[t] / 1000) * dt
    S2[t + 1] = S2[t] + (-S2[t] / tau_NMDA + (1 - S2[t]) * gamma * r2[t] / 1000) * dt

# 设置滑动窗口和步长
time_window = int(50 / dt)  # 时间窗口大小 (50 ms)
sliding_step = int(5 / dt)  # 滑动步长 (5 ms)

r1_smooth = [np.mean(r1[:time_window])]
r2_smooth = [np.mean(r2[:time_window])]
S1_smooth = [np.mean(S1[:time_window])]
S2_smooth = [np.mean(S2[:time_window])]
for t in range(1, int(np.floor((n_time_steps - time_window) / sliding_step)) + 1):
    start_idx = sliding_step * (t - 1)
    end_idx = start_idx + time_window
    r1_smooth.append(np.mean(r1[start_idx:end_idx]))
    r2_smooth.append(np.mean(r2[start_idx:end_idx]))
    S1_smooth.append(np.mean(S1[start_idx:end_idx]))
    S2_smooth.append(np.mean(S2[start_idx:end_idx]))
r1_smooth = np.array(r1_smooth)
r2_smooth = np.array(r2_smooth)
S1_smooth = np.array(S1_smooth)
S2_smooth = np.array(S2_smooth)

In [8]:
# Population 1 放电率轨迹
trace_r1 = go.Scatter(
    x=np.arange(dt * time_window, dt * n_time_steps, dt * sliding_step),
    y=r1_smooth,
    mode="lines",
    name="Firing Rate Population 1",
    line=dict(color="blue", width=2)
)

# Population 2 放电率轨迹
trace_r2 = go.Scatter(
    x=np.arange(dt * time_window, dt * n_time_steps, dt * sliding_step),
    y=r2_smooth,
    mode="lines",
    name="Firing Rate Population 2",
    line=dict(color="red", width=2)
)

# 刺激时期的高亮区域
stim_shape = dict(
    type="rect",
    xref="x",
    yref="paper",
    x0=stim_start,
    y0=0,
    x1=stim_end,
    y1=1,
    fillcolor="rgba(0, 255, 0, 0.1)",
    line=dict(width=0),
    layer="below"
)

# 时间序列图布局
layout_time_series = go.Layout(
    title="Wong and Wang Model Dynamics Over Time",
    xaxis=dict(title="Time (ms)"),
    yaxis=dict(title="Firing Rate (Hz)"),
    shapes=[stim_shape],  
    height=400,
    showlegend=True
)

fig_time_series = go.Figure(data=[trace_r1, trace_r2], layout=layout_time_series)
fig_time_series.show()

# S1 vs S2 轨迹
trace_phase_plane = go.Scatter(
    x=S1_smooth,
    y=S2_smooth,
    mode="lines+markers",
    marker=dict(
        color=np.arange(dt * time_window, dt * n_time_steps, dt * sliding_step),  # 使用时间编码颜色
        colorscale="Viridis",
        size=5,
        showscale=True,
        colorbar=dict(title="Time (ms)")
    ),
    line=dict(color="gray"),
    name="Trajectory"
)
layout_phase_plane = go.Layout(
    title="Phase-Plane Plot of Network Dynamics (S1 vs S2)",
    xaxis=dict(title="Synaptic Gating Variable S1", range=[0.0, 1.0]),
    yaxis=dict(title="Synaptic Gating Variable S2", range=[0.0, 1.0]),
    width=600,
    height=500,
    showlegend=True
)
fig_phase_plane = go.Figure(data=[trace_phase_plane], layout=layout_phase_plane)
fig_phase_plane.show()

完善绘图

In [6]:
# 不同的运动强度条件
motion_strengths = [51.2, 12.8, 0.0]  # 运动强度条件
colors = ["red", "orange", "blue"]  # 为不同强度设置颜色
linestyles = ["solid", "solid", "solid"]  # 为不同强度设置线型

# 储存每种运动强度下的放电率
r1_conditions = []
r2_conditions = []

# 模拟不同的运动强度条件
for i, coherence in enumerate(motion_strengths):
    # 重新设置刺激输入
    I_stim_1 = J_Input * mu0 * (1 + coherence / 100)
    I_stim_2 = J_Input * mu0 * (1 - coherence / 100)
    I_ext1 = np.full(n_time_steps, I_0)
    I_ext2 = np.full(n_time_steps, I_0)
    I_ext1[stim_indices] += I_stim_1
    I_ext2[stim_indices] += I_stim_2

    # 重置变量
    S1 = np.zeros(n_time_steps)
    S2 = np.zeros(n_time_steps)
    r1 = np.zeros(n_time_steps)
    r2 = np.zeros(n_time_steps)
    S1[0] = S2[0] = 0.1

    # 网络模拟
    for t in range(n_time_steps - 1):
        I_noise1 += dt / tau_AMPA * (-I_noise1) + np.sqrt(dt / tau_AMPA) * sigma_I * np.random.randn()
        I_noise2 += dt / tau_AMPA * (-I_noise2) + np.sqrt(dt / tau_AMPA) * sigma_I * np.random.randn()
        I1[t] = J_N * S1[t] - J_N_cross * S2[t] + I_ext1[t] + I_noise1
        I2[t] = J_N * S2[t] - J_N_cross * S1[t] + I_ext2[t] + I_noise2
        r1[t] = max(0, H(I1[t]))
        r2[t] = max(0, H(I2[t]))
        S1[t + 1] = S1[t] + (-S1[t] / tau_NMDA + (1 - S1[t]) * gamma * r1[t] / 1000) * dt
        S2[t + 1] = S2[t] + (-S2[t] / tau_NMDA + (1 - S2[t]) * gamma * r2[t] / 1000) * dt

    # 滑动窗口平滑
    r1_smooth = [np.mean(r1[:time_window])]
    r2_smooth = [np.mean(r2[:time_window])]
    for t in range(1, int(np.floor((n_time_steps - time_window) / sliding_step)) + 1):
        start_idx = sliding_step * (t - 1)
        end_idx = start_idx + time_window
        r1_smooth.append(np.mean(r1[start_idx:end_idx]))
        r2_smooth.append(np.mean(r2[start_idx:end_idx]))
    r1_conditions.append(r1_smooth)
    r2_conditions.append(r2_smooth)

# --- 绘制放电率轨迹 ---
fig = go.Figure()

# 左图（motion on 到 eye movement 前）
for i, motion_strength in enumerate(motion_strengths):
    fig.add_trace(go.Scatter(
        x=np.arange(dt * time_window, dt * n_time_steps, dt * sliding_step),
        y=r1_conditions[i],
        mode="lines",
        name=f"Motion Strength {motion_strength}",
        line=dict(color=colors[i], width=2, dash=linestyles[i])
    ))

# 添加刺激高亮区域
fig.update_layout(
    title="Firing Rate Dynamics with Motion Strength",
    xaxis=dict(title="Time (ms)"),
    yaxis=dict(title="Firing Rate (sp/s)"),
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="paper",
            x0=stim_start,
            y0=0,
            x1=stim_end,
            y1=1,
            fillcolor="rgba(0, 255, 0, 0.1)",
            line=dict(width=0),
            layer="below"
        )
    ],
    template="plotly_white"
)

fig.show()

加入决策机制

In [9]:
# Decision thresholds
decision_threshold = 15.0  
min_rate_difference = 5.0  

# Coherence levels and trials
coherence_levels = np.array([-0.512, -0.256, -0.128, -0.064, -0.032, 0, 0.032, 0.064, 0.128, 0.256, 0.512])
n_trials_per_coherence = 100
choices = np.zeros((len(coherence_levels), n_trials_per_coherence))
decision_times = np.zeros((len(coherence_levels), n_trials_per_coherence))

for c_idx, coherence in enumerate(coherence_levels):
    for trial in range(n_trials_per_coherence):
        S1 = np.zeros(n_time_steps)  
        S2 = np.zeros(n_time_steps)  
        r1 = np.zeros(n_time_steps)  
        r2 = np.zeros(n_time_steps)  
        I1 = np.zeros(n_time_steps)  
        I2 = np.zeros(n_time_steps)  

        I_stim_1 = J_Input * mu0 * (1 + coherence)
        I_stim_2 = J_Input * mu0 * (1 - coherence)
        I_ext1 = np.full(n_time_steps, I_0)
        I_ext2 = np.full(n_time_steps, I_0)
        I_ext1[stim_indices] += I_stim_1
        I_ext2[stim_indices] += I_stim_2

        I_noise1 = sigma_I * np.random.randn()
        I_noise2 = sigma_I * np.random.randn()
        S1[0] = S2[0] = 0.1

        decision_made = False
        decision_time = None
        choice = None

        for t in range(n_time_steps - 1):
            I_noise1 += dt / tau_AMPA * (-I_noise1) + np.sqrt(dt / tau_AMPA) * sigma_I * np.random.randn()
            I_noise2 += dt / tau_AMPA * (-I_noise2) + np.sqrt(dt / tau_AMPA) * sigma_I * np.random.randn()

            # Compute total input currents
            I1[t] = J_N * S1[t] - J_N_cross * S2[t] + I_ext1[t] + I_noise1
            I2[t] = J_N * S2[t] - J_N_cross * S1[t] + I_ext2[t] + I_noise2

            r1[t] = H(I1[t])
            r2[t] = H(I2[t])
            r1[t] = max(0, r1[t])
            r2[t] = max(0, r2[t])

            S1[t + 1] = S1[t] + (-S1[t] / tau_NMDA + (1 - S1[t]) * gamma * r1[t] / 1000) * dt
            S2[t + 1] = S2[t] + (-S2[t] / tau_NMDA + (1 - S2[t]) * gamma * r2[t] / 1000) * dt

            if not decision_made and time[t] >= stim_start:
                if r1[t] - r2[t] >= min_rate_difference and r1[t] >= decision_threshold:
                    decision_made = True
                    decision_time = time[t] - stim_start
                    choice = 1
                elif r2[t] - r1[t] >= min_rate_difference and r2[t] >= decision_threshold:
                    decision_made = True
                    decision_time = time[t] - stim_start
                    choice = 2
            if decision_made:
                break

        if decision_made:
            choices[c_idx, trial] = choice
            decision_times[c_idx, trial] = decision_time
        else:
            decision_times[c_idx, trial] = np.nan
            choices[c_idx, trial] = 0 

proportion_choice1 = np.zeros(len(coherence_levels))
mean_decision_times = np.zeros(len(coherence_levels))
std_decision_times = np.zeros(len(coherence_levels))

for c_idx in range(len(coherence_levels)):
    choices_c = choices[c_idx, :]
    decision_times_c = decision_times[c_idx, :]
    valid_trials = choices_c != 0  

    proportion_choice1[c_idx] = np.sum(choices_c[valid_trials] == 1) / np.sum(valid_trials)

    mean_decision_times[c_idx] = np.nanmean(decision_times_c[valid_trials])
    std_decision_times[c_idx] = np.nanstd(decision_times_c[valid_trials])

psychometric_curve = go.Scatter(
    x=coherence_levels,  
    y=proportion_choice1,  
    mode='lines+markers',  
    name='Psychometric Curve',  
    line=dict(color='blue', width=2),  
    marker=dict(size=8)  
)

layout_psychometric = go.Layout(
    title='Psychometric Curve',  
    xaxis=dict(title='Coherence Level'),  
    yaxis=dict(title='Proportion Choosing Population 1'),  
    template='plotly_white'  
)

fig_psychometric = go.Figure(data=[psychometric_curve], layout=layout_psychometric)
fig_psychometric.show()

chronometric_curve = go.Scatter(
    x=coherence_levels,  
    y=mean_decision_times,  
    mode='lines+markers',  
    name='Chronometric Curve',  
    line=dict(color='red', width=2),  
    marker=dict(size=8)  
)


layout_chronometric = go.Layout(
    title='Chronometric Curve',  
    xaxis=dict(title='Coherence Level'),  
    yaxis=dict(title='Mean Decision Time (ms)'),  
    template='plotly_white'  
)

fig_chronometric = go.Figure(data=[chronometric_curve], layout=layout_chronometric)
fig_chronometric.show()